In [1]:
import data
from data import DataLoader
import importlib
importlib.reload(data)
import torch
import torch.nn as nn
import torch.nn.functional as F

from gpt import GPTConfig

In [ ]:
data = DataLoader()
config = GPTConfig(vocab_size=50257, block_size=1024,
                   n_layer=12, n_head=12, n_embd=768)


In [18]:
print(data.inputs[0])
print(data.outputs[0])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13, 198, 198, 5962, 22307, 25, 198, 1639, 389, 477, 12939, 2138, 284, 4656, 621, 284, 1145, 680, 30, 198, 198, 3237, 25, 198, 4965, 5634, 13, 12939, 13, 198, 198, 5962, 22307, 25, 198, 5962, 11, 345, 760, 327, 1872, 385, 1526, 28599, 318, 4039, 4472, 284, 262, 661, 13, 198, 198, 3237, 25, 198, 1135, 760, 470, 11, 356, 760, 470, 13, 198, 198, 5962, 22307, 25, 198, 5756, 514, 1494, 683, 11, 290, 356, 1183, 423, 11676, 379, 674, 898, 2756, 13, 198, 3792, 470, 257, 15593, 30, 198, 198, 3237, 25, 198, 2949, 517, 3375, 319, 470, 26, 1309, 340, 307, 1760, 25, 1497, 11, 1497, 0, 198, 198, 12211, 22307, 25, 198, 3198, 1573, 11, 922, 4290, 13, 198, 198, 5962, 22307, 25, 198, 1135, 389, 17830, 3595, 4290, 11, 262, 1458, 1173, 1547, 922, 13, 198, 2061, 4934, 969, 5036, 896, 319, 561, 26958, 514, 25, 611, 484, 198, 19188, 7800, 514, 475, 262, 48713, 414, 11, 981, 340, 547, 198, 

In [74]:
one_tri = torch.tril(torch.ones(4,4))
print(one_tri)
m = torch.randint(0, 10, (4,4)).float()
print(m)

masked = m.masked_fill(one_tri==0, float('-inf'))
print(masked)

tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[7., 4., 1., 7.],
        [5., 8., 4., 0.],
        [0., 6., 3., 1.],
        [8., 4., 4., 0.]])
tensor([[7., -inf, -inf, -inf],
        [5., 8., -inf, -inf],
        [0., 6., 3., -inf],
        [8., 4., 4., 0.]])


In [2]:
from qadata import *

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
qa_loader = QADataLoader(
    dataset_name="databricks/databricks-dolly-15k",
    batch_size=batch_size,
    block_size=block_size
)

# Test getting a batch (old style)
x, y = qa_loader.get_batch('train')
print(f"Batch shapes - Input: {x.shape}, Target: {y.shape}")
print(f"Vocabulary size: {vocab_size}")

# Decode and show a sample
sample_input = tokenizer.decode(x[0].tolist())
print(f"\nSample input (truncated):\n{sample_input[:200]}...")

# Modern training loop example
print("\n\nModern training loop example:")
train_loader, val_loader = qa_loader.get_loaders()

for i, (inputs, targets) in enumerate(train_loader):
    inputs = inputs.to(device)
    targets = targets.to(device)
    
    print(f"Batch {i}: Input shape {inputs.shape}, Target shape {targets.shape}")
    print(inputs[0], targets[0])
    
    if i >= 1:  # Just show first 3 batches
        break

Loading dataset: databricks/databricks-dolly-15k
train dataset size: 13487 examples
val dataset size: 1501 examples


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batch shapes - Input: torch.Size([8, 255]), Target: torch.Size([8, 255])
Vocabulary size: 32000

Sample input (truncated):
<s> List 3 cities on the east coast of Australia. Sydney, Brisbane and Newcastle are Australian cities located on the east coast of Australia.</s></s></s></s></s></s></s></s></s></s></s></s></s></s></...


Modern training loop example:


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batch 0: Input shape torch.Size([8, 255]), Target shape torch.Size([8, 255])
tensor([    1,  3529,  2367,   592,   263, 11473,  4955,   310,   278, 25200,
         8068,  9538, 29973, 20628,   727,   526, 29871, 29906, 29947,   970,
         9562,   297, 25200,   541, 29892,   408,   278,   937,   304,  1722,
          297,   278, 17780,  7483, 29892,   278, 11265,   310,  8068,  9538,
          471,  5220,   287,   411, 15151, 29945, 29900, 29892, 29900, 29900,
        29900,   491,  8578,  9716,  1336,   391, 11571,  1704, 10052,   347,
        29889,  2180,   278,  8718, 25672,   263,  4382,  1393,   515,  1704,
        10052,   347,   471,  1303,   714, 23659, 29901,   376,  4806,  9311,
          393,   445,  9538,   338,   304,  6548,   297,  5407,  2264,  1629,
         1156,  1629, 29892,   322,  6356,   697,   310,   278,  1556,  3104,
          296,   946, 15942,   363,   278,  1781,   310,   278,  2305,   363,
          599,   931,   304,  2041,  1213,    13,    13,  1576,  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
